In [1]:
import pandas as pd
import numpy as np
import math
import statistics
import glob
import tqdm

In [2]:
listOfYearlyImportFiles = glob.glob('Data/lab_data/*')
print(len(listOfYearlyImportFiles))
print(listOfYearlyImportFiles)

15
['Data/lab_data/req1512_lab_2020.csv', 'Data/lab_data/req1512_lab_2006.csv', 'Data/lab_data/req1512_lab_2014.csv', 'Data/lab_data/req1512_lab_2010.csv', 'Data/lab_data/req1512_lab_2015.csv', 'Data/lab_data/req1512_lab_2018.csv', 'Data/lab_data/req1512_lab_2017.csv', 'Data/lab_data/req1512_lab_2019.csv', 'Data/lab_data/req1512_lab_2016.csv', 'Data/lab_data/req1512_lab_2012.csv', 'Data/lab_data/req1512_lab_2008.csv', 'Data/lab_data/req1512_lab_2011.csv', 'Data/lab_data/req1512_lab_2007.csv', 'Data/lab_data/req1512_lab_2013.csv', 'Data/lab_data/req1512_lab_2009.csv']


In [3]:
componentDataDF = pd.read_csv('Data/211209_Lab_Crosswalk_YN.csv')
componentDataDF.head()

,First Alphabetical Component,Related Components 0,Related Components 1,Related Components 2,Related Components 3,Comments
0,% SATURATION,NaN,NaN,NaN,NaN,Need to see data
1,% SATURATION (FE),IRON SATURATION,NaN,NaN,NaN,NaN
2,% SATURATION TIBC,NaN,NaN,NaN,NaN,Need to see data
3,25-HYDROXY D TOTAL,"VIT D, 25-HYDROXY","VITAMIN D, 25 OH TOTAL","VITAMIN D, 25-HYDROXY",NaN,NaN
4,A/G RATIO,ALB/GLOBULIN RATIO,ALBUMIN/GLOB SERPL,ALBUMIN/GLOBULIN RATIO,NaN,NaN


In [6]:
listOfLabComponents = []
for eachIndex, eachRow in componentDataDF.iterrows():
    listOfLabComponents.append(eachRow['First Alphabetical Component'])
    for eachColumnName in ['Related Components 0', 'Related Components 1', 'Related Components 2', 'Related Components 3']:
        if isinstance(eachRow[eachColumnName], str):
            listOfLabComponents.append(eachRow[eachColumnName])

print('number of lab components:', len(listOfLabComponents))
print(listOfLabComponents)

number of lab components: 300
['% SATURATION', '% SATURATION (FE)', 'IRON SATURATION', '% SATURATION TIBC', '25-HYDROXY D TOTAL', 'VIT D, 25-HYDROXY', 'VITAMIN D, 25 OH TOTAL', 'VITAMIN D, 25-HYDROXY', 'A/G RATIO', 'ALB/GLOBULIN RATIO', 'ALBUMIN/GLOB SERPL', 'ALBUMIN/GLOBULIN RATIO', 'ABSOLUTE LYMPHOCYTES', 'LYMPHOCYTE', 'LYMPHS', 'ABSOLUTE PLASMA CELLS', 'PLASMA CELL ABS', 'ABSOLUTE REACTIVE LYMPHOCYTES', 'LYMPHS REACTIVE', 'REAC LYMPHS ABS', 'ACANTHOCYTES', 'ACANTHOCYTES NUMERIC', 'ALBUMIN', 'ALK PHOS', 'ALK PHOSPHATASE', 'ALKALINE PHOSPHATASE', 'ALKALINE PHOSPHATASE, S', 'OSH ALP SERUM', 'ALT', 'ALT (SGPT)', 'OSH ALT SERUM', 'AMIKACIN TR', 'AMYLASE', 'ANION GAP', 'ANISOCYTOSIS', 'AST', 'OSH AST SERUM', 'B-TYPE NATRIURETIC PEPTIDE', 'BNP', 'BANDS', 'BANDS ABS', 'BANDS PCT', 'BASO (%) AUTO', 'BASO%(AUTO)', 'BASOPHILS, %', 'BASOS PCT', 'BASO # AUTO', 'BASO#(AUTO)', 'BASOPHILS # BLD AUTO', 'BASOS ABS', 'BASO STIPPLING', 'BASOPHIL', 'BILIRUBIN', 'BILIRUBIN, TOTAL', 'OSH TBIL SERUM', 'TOT

In [15]:
listOfLabComponents[174]

'LYMPH%(AUTO)'

In [16]:
listOfLabComponents=listOfLabComponents[174:]

In [17]:
# function to preprocess list to remove nonsensical inputs and get only numerical values
def filterList(oldList):
    stripedList = [x.strip('<>') for x in oldList]
    filteredList = []
    for eachItem in stripedList:
        try:
            floatItem = float(eachItem)
            filteredList.append(floatItem)
        except:
            pass
    noNanList = [x for x in filteredList if math.isnan(x) == False]
    return noNanList

# get basic parameters of list like min, max, mean, standard deviation
def analyzeList(thisList):
    try:
        thisMin = min(thisList)
    except:
        thisMin = -1
        
    try:
        thisMax = max(thisList)
    except:
        thisMax = -1
        
    try:
        thisMean = statistics.mean(thisList)
    except:
        thisMean = -1
        
    try:
        thisStdev = statistics.stdev(thisList)
    except:
        thisStdev = -1
        
    return {'count':len(thisList),'min':thisMin, 'max':thisMax, 'mean':thisMean, 'stdev':thisStdev}

# generate string
def genString(componentName, paramDict):
    thisString = componentName.strip(',')
    thisString +=','+str(paramDict['count'])
    thisString +=','+str(paramDict['min'])
    thisString +=','+str(paramDict['max'])
    thisString +=','+str(paramDict['mean'])
    thisString +=','+str(paramDict['stdev'])
    thisString +='\n'
    return thisString

In [ ]:
# For EDA, to figure out which column was the ex
with open("outputs/results.csv", "a") as text_file:
    text_file.write('component name,count,min,max,mean,stdev\n')

for eachComponent in listOfLabComponents:
    print(eachComponent)
    completeListOfORDvalues = []
    for eachYearlyImportFile in listOfYearlyImportFiles:
        workingDataFrame = pd.read_csv(eachYearlyImportFile, usecols = ['COMMON_NAME','COMPONENT_NAME','BASE_NAME','ORD_VALUE','REFERENCE_UNIT'])
        workingDataFrame['ORD_VALUE'] = workingDataFrame['ORD_VALUE'].astype('str') 
        workingDataFrame['nameMatched'] = workingDataFrame.apply(lambda x: x['COMMON_NAME']==eachComponent or x['COMPONENT_NAME']==eachComponent, axis=1)
        workingDataFrame = workingDataFrame[workingDataFrame['nameMatched']==True]
        eachYearListOfORDvalues = workingDataFrame['ORD_VALUE'].values
        eachYearListOfORDvalues = filterList(eachYearListOfORDvalues)
        print(len(eachYearListOfORDvalues))
        completeListOfORDvalues += eachYearListOfORDvalues
        print(len(completeListOfORDvalues))
    with open("outputs/results.csv", "a") as text_file:
        stringToSave = genString(eachComponent,analyzeList(completeListOfORDvalues))
        text_file.write(stringToSave)

LYMPH%(AUTO)
0
0
0
0
10
10
0
10
6
16
5
21
0
21
2
23
5
28
22
50
0
50
8
58
0
58
10
68
0
68
LYMPHOCYTE %
0
0
0
0
1
1
0
1
2
3
1
4
0
4
0
4
0
4
2
6
0
6
0
6
0
6
0
6
0
6
LYMPHOCYTES, %
4
4
1
5
1
6
6
12
10
22
3
25
4
29
0
29
5
34
2
36
8
44
11
55
9
64
1
65
8
73
LYMPHS PCT
304735
304735
37645
342380
214486
556866
149427
706293
238313
944606
291557
1236163
266355
1502518
327560
1830078
249589
2079667
178021
2257688
281265
2538953
158309
2697262
47505
2744767
196451
2941218
143274
3084492
LYMPH # AUTO
3
3
0
3
0
3
0
3
0
3
0
3
0
3
3
6
0
6
0
6
0
6
0
6
0
6
0
6
0
6
LYMPH#(AUTO)
0
0
0
0
10
10
0
10
5
15
5
20
0
20
2
22
5
27
23
50
0
50
8
58
0
58
10
68
0
68
LYMPHOCYTES # BLD AUTO
0
0
0
0
1
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
LYMPHS (ABSOLUTE)
2
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
LYMPHS ABS
285781
285781
33927
319708
203745
523453
146421
669874
221185
891059
283647
1174706
260626
1435332
315660
1750992
239223
1990215
172146
2162361
277884
2440245
153857
2594102
44517
263861